# LLM Calls

In [8]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
# llm = OpenAI(model="gpt-3.5-turbo")
chat_model = ChatOpenAI(model="gpt-3.5-turbo")
# llm.predict("hi!")
chat_model.predict("Hi")

'Hello! How can I help you today?'

# Download competitions

In [15]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [17]:
# api.dataset_download_file('kaggle/meta-kaggle/download?datasetVersionNumber=1131',
#                           file_name='Competitions.csv',
#                           path='./data')

api.dataset_download_file('kaggle/meta-kaggle/download?datasetVersionNumber=1131',
                          file_name='ForumTopics.csv',
                          path='./data')

True

In [21]:
import zipfile

def extract_zip(name):
    zip_path = f'./data/{name}.csv.zip'  
    extract_path = './data'  

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

extract_zip("ForumTopics")

In [23]:
import pandas as pd

In [50]:
forums_df = pd.read_csv("./data/ForumTopics.csv")
forums_df.head()

,Id,ForumId,KernelId,LastForumMessageId,FirstForumMessageId,CreationDate,LastCommentDate,Title,IsSticky,TotalViews,Score,TotalMessages,TotalReplies
0,46,1,NaN,684511.0,220.0,08/03/2010 22:24:40,11/30/2019 00:07:53,The solution,False,11409,0,8,7
1,6589,1,NaN,554419.0,36135.0,12/12/2013 22:18:34,06/17/2019 11:42:33,What is H in test-data?,False,1940,0,3,2
2,36,1,NaN,141.0,134.0,06/30/2010 20:20:10,06/30/2010 20:20:10,Scoring system,False,4781,0,4,3
3,119608,1,NaN,1667283.0,684518.0,11/30/2019 00:23:31,01/28/2022 12:52:13,Submissions Closed/Solution Data Unavailable,False,0,0,5,4
4,62,1,NaN,402.0,328.0,08/11/2010 18:12:05,08/11/2010 18:12:05,Accuracy on full data set,False,6241,0,3,2


In [51]:
comp_df = pd.read_csv("./data/Competitions.csv")
# we can filter to keep only the best competitions, i.e. "Featured", "Reseach" and recruitments

In [17]:

comp_slug = "Eurovision2010"
comp_url = "https://www.kaggle.com/c/"+str(comp_slug)
comp_url

'https://www.kaggle.com/c/Eurovision2010'

# Scrapping a competition

In [22]:
from langchain.document_loaders import AsyncHtmlLoader
urls = [comp_url]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.34it/s]


In [24]:
docs[0]

Document(page_content='\r\n\r\n<!DOCTYPE html>\r\n<html lang="en">\r\n\r\n<head>\r\n  <title>Forecast Eurovision Voting  | Kaggle</title>\r\n  <meta charset="utf-8" />\r\n    <meta name="robots" content="index, follow" />\r\n  <meta name="description" content="This competition requires contestants to forecast the voting for this year&#x27;s Eurovision Song Contest in Norway on May 25th, 27th and 29th. " />\r\n  <meta name="turbolinks-cache-control" content="no-cache" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0">\r\n  <meta name="theme-color" content="#008ABC" />\r\n  <script nonce="53d9Z7YCloSoXWbCeMIDJQ==" type="text/javascript">\r\n    window["pageRequestStartTime"] = 1695309355526;\r\n    window["pageRequestEndTime"] = 1695309355561;\r\n    window["initialPageLoadStartTime"] = new Date().getTime();\r\n  </script>\r\n  <link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous" /><link

In [37]:
from langchain.document_loaders import AsyncChromiumLoader
import nest_asyncio
nest_asyncio.apply() # for use in notebook

loader = AsyncChromiumLoader(["http://www.kaggle.com/c/Eurovision2010"])
html = loader.load()
html

[Document(page_content='Error: Protocol error (Page.navigate): Cannot navigate to invalid URL\n=========================== logs ===========================\nnavigating to "www.kaggle.com/c/Eurovision2010", waiting until "load"\n============================================================', metadata={'source': 'www.kaggle.com/c/Eurovision2010'})]

In [34]:
html

[Document(page_content='Error: net::ERR_CERT_AUTHORITY_INVALID at https://www.kaggle.com/c/Eurovision2010\n=========================== logs ===========================\nnavigating to "https://www.kaggle.com/c/Eurovision2010", waiting until "load"\n============================================================', metadata={'source': 'https://www.kaggle.com/c/Eurovision2010'})]

In [51]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context(ignore_https_errors=True)
        page = await context.new_page()
        await page.goto('http://www.kaggle.com/c/Eurovision2010')
        results = await page.content()
        await browser.close()
        print(results)

asyncio.run(main())

<!DOCTYPE html><html lang="en"><head>
  <title>Forecast Eurovision Voting  | Kaggle</title>
  <meta charset="utf-8">
    <meta name="robots" content="index, follow">
  <meta name="description" content="This competition requires contestants to forecast the voting for this year's Eurovision Song Contest in Norway on May 25th, 27th and 29th. ">
  <meta name="turbolinks-cache-control" content="no-cache">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0">
  <meta name="theme-color" content="#008ABC">
  <script src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.IoxrLNdlTyI.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo9N48n3oloz8UTxoCozKcpUKaADkg/cb=gapi.loaded_0?le=scs" nonce="" async=""></script><script nonce="" type="text/javascript">
    window["pageRequestStartTime"] = 1695312806729;
    window["pageRequestEndTime"] = 1695312807320;
    window["initialPageLoadStartTime"] = new Date().getTime();
  </script>
  <link

In [67]:
from selenium import webdriver
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument("ignore-certificate-errors")
options.add_argument("--headless")
driver = webdriver.Chrome(options=options) 
driver.get('http://www.kaggle.com/c/Eurovision2010')

driver.implicitly_wait(10)
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument("ignore-certificate-errors")
options.add_argument("--headless")
driver = webdriver.Chrome(options=options) 
driver.get('http://www.kaggle.com/c/Eurovision2010')

element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "pageheader-nav-item--overview"))
)

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [10]:
soup

<html lang="en"><head>
<title>Forecast Eurovision Voting  | Kaggle</title>
<meta charset="utf-8"/>
<meta content="index, follow" name="robots"/>
<meta content="This competition requires contestants to forecast the voting for this year's Eurovision Song Contest in Norway on May 25th, 27th and 29th. " name="description"/>
<meta content="no-cache" name="turbolinks-cache-control"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0" name="viewport"/>
<meta content="#008ABC" name="theme-color"/>
<script async="" nonce="" src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.en.L92w_vMR5kE.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo-GjmyR8TQVDKPrbqj5UQg7cU3cCA/cb=gapi.loaded_0?le=scs"></script><script nonce="" type="text/javascript">
    window["pageRequestStartTime"] = 1695550278940;
    window["pageRequestEndTime"] = 1695550279021;
    window["initialPageLoadStartTime"] = new Date().getTime();
  </script>
<link crossorigin="anonymous" href="

In [11]:
soup.get_text()

"\nForecast Eurovision Voting  | Kaggle\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ncodeNew Notebooktable_chartNew Datasetemoji_eventsNew CompetitionNo Active EventsCreate notebooks and keep track of their status here.addNew Notebookauto_awesome_motion00 Active Eventsexpand_morebookmark_borderBookmarkmenuSkip tocontentCreatesearchexploreHomeemoji_eventsCompetitionstable_chartDatasetstenancyModelscodeCodecommentDiscussionsschoolLearnexpand_moreMoreauto_awesome_motion0View Active EventsmenuSkip tocontentsearchSign InRegisterWe use cookies on Kaggle to deliver our services, analyze web traffic, and improve your experience on the site. By using Kaggle, you agree to our use of cookies.Got itLearn moreFeatured Prediction CompetitionForecast Eurovision Voting This competition requires contestants to forecast the voting for this year's Eurovision Song Contest in Norway on May 25th, 27th and 29th. $1,000Prize Money22 teams13 years agoOverviewDataCodeDiscussionRulesm

# Format competition summary and problem

In [12]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Here is data scrapped from kaggle competition. Please format your response this way, and add as much details as you can:
1. Explain the problem with relevant details for a machine learning practitionner. 
2. what is the machine learning problem (is it binary classification ? A regression ? Clustering ?:
{competition_text}""")


chat_model.predict(prompt.format(competition_text=soup.get_text()))

"1. The problem presented in this Kaggle competition is to forecast the voting for the Eurovision Song Contest in Norway. The Eurovision Song Contest is a major event that involves participants from various countries, and each country has a panel of judges who vote for their favorite performances. To be successful in this competition, a machine learning practitioner needs to understand the underlying factors that influence voting patterns in the Eurovision Song Contest. They would also need to analyze historical voting data, as well as any other relevant features, to develop accurate models that can predict the voting outcomes for the contest.\n\n2. The machine learning problem in this competition can be considered as a regression task. The goal is to predict the voting results, which are numerical values representing the number of votes received by each participant. The machine learning model will need to analyze various features, such as the participants' previous performances, the c

# Get solutions URL for a competition

In [77]:
def check_solution(topic):
    is_solution = False
    to_exclude = ["?", "submit", "why", "what", "resolution", "benchmark"]
    if "solution" in topic.lower():
        is_solution = True
        for exc in to_exclude:
            if exc in topic.lower():
                is_solution = False
    to_include = ["2nd place code", '"dance with ensemble" sharing']
    for inc in to_include:
        if inc in topic.lower():
            is_solution = True
    return is_solution

def check_solution_llm(topic_title, comp_subject):
    prompt = PromptTemplate.from_template("""Here is the title of a discussion of a kaggle whose subject is {subject}. 
                                          Tell me, from the title, if this discussion is going to be an explanation of a solution, or not.
                                          Output only 0 or 1
                                          {title}
                                          """)
    return chat_model.predict(prompt.format(subject=comp_subject, title=topic_title))

def get_discussion_results(forum_id, n):
    results_df = forums_df[forums_df["ForumId"]==forum_id]
    results_df["is_solution"] = results_df["Title"].apply(lambda x: check_solution(str(x)))
    results_df = results_df[results_df["is_solution"] == 1]
    results_df = results_df.sort_values(by=["Score","TotalMessages"], ascending=False).head(n).reset_index(drop=True)
    return results_df[["Title", "Id", "Score", "TotalMessages", "TotalReplies"]]

In [78]:
comp_id = 5603
id_forum_comp = comp_df.iloc[comp_id]["ForumId"]
comp_slug = comp_df.iloc[comp_id]["Slug"]

discussions_df = get_discussion_results(id_forum_comp, 5)
best_forum = discussions_df.iloc[0]["Id"]

url = f"https://www.kaggle.com/c/{comp_slug}/discussion/{best_forum}"
url

/tmp/ipykernel_8112/2344154697.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df["is_solution"] = results_df["Title"].apply(lambda x: check_solution(str(x)))


'https://www.kaggle.com/c/playground-series-s3e16/discussion/416766'

In [69]:
discussions_df

,Title,Id,Score,TotalMessages,TotalReplies
0,Our approach,6321,23,33,32
1,Beating The Benchmark In Python (2230k MAE),5282,22,48,47
2,Moar starter code and a NetCDF4 tutorial,5716,13,12,11
3,Plot of the day,5619,11,8,7
4,Calc distances between Lon/Lat on a sphere,5074,4,6,5


In [75]:
comp_df[comp_df["HostSegmentTitle"] == "Playground"]

,Id,Slug,Title,Subtitle,HostSegmentTitle,ForumId,OrganizationId,EnabledDate,DeadlineDate,ProhibitNewEntrantsDeadlineDate,...,CanQualifyTiers,TotalTeams,TotalCompetitors,TotalSubmissions,ValidationSetName,ValidationSetValue,EnableSubmissionModelHashes,EnableSubmissionModelAttachments,HostName,CompetitionTypeId
101,3362,dogs-vs-cats,Dogs vs. Cats,Create an algorithm to distinguish dogs from cats,Playground,310,4.0,09/25/2013 14:44:03,02/01/2014 23:59:00,NaN,...,True,213,232,1218,NaN,NaN,False,False,NaN,1
132,3535,learning-social-circles,Learning Social Circles in Networks,Model friend memberships to multiple circles,Playground,449,4.0,05/06/2014 20:27:23,10/28/2014 23:59:00,10/21/2014 23:59:00,...,True,202,230,1751,NaN,NaN,False,False,NaN,1
136,3586,crowdflower-weather-twitter,Partly Sunny with a Chance of Hashtags,What can a #machine learn from tweets about th...,Playground,314,3.0,09/27/2013 14:43:09,12/01/2013 23:59:00,NaN,...,True,258,299,3590,NaN,NaN,False,False,NaN,1
142,3638,conway-s-reverse-game-of-life,Conway's Reverse Game of Life,Reverse the arrow of time in the Game of Life,Playground,323,4.0,10/14/2013 20:18:56,03/02/2014 23:59:00,NaN,...,True,141,177,1321,NaN,NaN,False,False,NaN,1
145,3649,cifar-10,CIFAR-10 - Object Recognition in Images,"Identify the subject of 60,000 labeled images",Playground,329,4.0,10/18/2013 21:07:23,10/18/2014 23:59:00,NaN,...,False,231,292,1632,NaN,NaN,False,False,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5603,51983,playground-series-s3e16,Regression with a Crab Age Dataset,"Playground Series - Season 3, Episode 16",Playground,3228958,4.0,05/30/2023 00:00:29,06/12/2023 23:59:00,NaN,...,False,1429,1467,10531,NaN,NaN,False,False,NaN,1
5608,53376,playground-series-s3e17,Binary Classification of Machine Failures,"Playground Series - Season 3, Episode 17",Playground,3326880,4.0,06/13/2023 00:04:18,06/26/2023 23:59:00,NaN,...,False,1502,1544,11708,NaN,NaN,False,False,NaN,1
5609,53377,playground-series-s3e18,Explore Multi-Label Classification with an Enz...,"Playground Series - Season 3, Episode 18",Playground,3326881,4.0,06/27/2023 00:04:53,07/10/2023 23:59:00,NaN,...,False,1047,1065,8801,NaN,NaN,False,False,NaN,1
5610,57094,playground-series-s3e19,Forecasting Mini-Course Sales,"Playground Series - Season 3, Episode 19",Playground,3553816,4.0,07/11/2023 00:17:58,07/31/2023 23:59:00,NaN,...,False,1172,1212,10058,NaN,NaN,False,False,NaN,1


# Scrapping of solution URL